# mixColumn()

In [3]:
def convert_hex_to_dec(data: str):
    return int(data, 16)

def convert_dec_to_hex(data: int):
    return hex(data)[2:].zfill(2)

def multiply__2(data: int):
    shifted = data << 1
    if data & 0x80:  # 8번째 비트가 1이면
        shifted ^= 0x1B
    return shifted & 0xFF  # 8비트 제한

def multiply__3(data: int):
    return multiply__2(data) ^ data

text = '0123456789abcdef0123456789abcdef'  # 32 hex → 16 bytes → 4열 (AES 상태행렬)
part_length = 8  # 각 열 8 hex (4바이트)
columns = [text[i:i+part_length] for i in range(0, len(text), part_length)]

out = ''

for col in columns:
    # 2자리씩 슬라이싱해서 s1~s4 추출 (총 4개 바이트)
    s1 = convert_hex_to_dec(col[0:2])
    s2 = convert_hex_to_dec(col[2:4])
    s3 = convert_hex_to_dec(col[4:6])
    s4 = convert_hex_to_dec(col[6:8])

    # MixColumns 열 연산 수행
    out_s1 = multiply__2(s1) ^ multiply__3(s2) ^ s3 ^ s4
    out_s2 = s1 ^ multiply__2(s2) ^ multiply__3(s3) ^ s4
    out_s3 = s1 ^ s2 ^ multiply__2(s3) ^ multiply__3(s4)
    out_s4 = multiply__3(s1) ^ s2 ^ s3 ^ multiply__2(s4)

    # 결과 연결
    out += convert_dec_to_hex(out_s1)
    out += convert_dec_to_hex(out_s2)
    out += convert_dec_to_hex(out_s3)
    out += convert_dec_to_hex(out_s4)

print("MixColumns 결과 (hex):", out)


MixColumns 결과 (hex): 45ef01abcd67892345ef01abcd678923


AES에서 **복호화용 MixColumns**, 즉 `InvMixColumns` 연산은 암호화 때 사용된 MixColumns의 **역함수**입니다. 구조는 거의 동일하지만, 사용하는 상수가 다릅니다.

---

## 🔁 AES MixColumns vs InvMixColumns (1열 기준)

| 출력 바이트 | 암호화(MixColumns) | 복호화(InvMixColumns) |
| --- | --- | --- |
| `s1'` | `2·s1 ⊕ 3·s2 ⊕ s3 ⊕ s4` | `14·s1 ⊕ 11·s2 ⊕ 13·s3 ⊕ 9·s4` |
| `s2'` | `s1 ⊕ 2·s2 ⊕ 3·s3 ⊕ s4` | `9·s1 ⊕ 14·s2 ⊕ 11·s3 ⊕ 13·s4` |
| `s3'` | `s1 ⊕ s2 ⊕ 2·s3 ⊕ 3·s4` | `13·s1 ⊕ 9·s2 ⊕ 14·s3 ⊕ 11·s4` |
| `s4'` | `3·s1 ⊕ s2 ⊕ s3 ⊕ 2·s4` | `11·s1 ⊕ 13·s2 ⊕ 9·s3 ⊕ 14·s4` |

In [4]:
def multiply(data: int, factor: int) -> int:
    result = 0
    for i in range(8):
        if factor & 1:
            result ^= data
        hi_bit = data & 0x80
        data = (data << 1) & 0xFF
        if hi_bit:
            data ^= 0x1B
        factor >>= 1
    return result

def inv_mix_single_column(s1, s2, s3, s4):
    o1 = multiply(s1, 14) ^ multiply(s2, 11) ^ multiply(s3, 13) ^ multiply(s4, 9)
    o2 = multiply(s1, 9)  ^ multiply(s2, 14) ^ multiply(s3, 11) ^ multiply(s4, 13)
    o3 = multiply(s1, 13) ^ multiply(s2, 9)  ^ multiply(s3, 14) ^ multiply(s4, 11)
    o4 = multiply(s1, 11) ^ multiply(s2, 13) ^ multiply(s3, 9)  ^ multiply(s4, 14)
    return o1, o2, o3, o4


In [5]:
# 앞서 MixColumns 했던 hex 블록 out 를 다시 복호화
restored = ''
for i in range(0, len(out), 8):
    s1 = convert_hex_to_dec(out[i:i+2])
    s2 = convert_hex_to_dec(out[i+2:i+4])
    s3 = convert_hex_to_dec(out[i+4:i+6])
    s4 = convert_hex_to_dec(out[i+6:i+8])

    r1, r2, r3, r4 = inv_mix_single_column(s1, s2, s3, s4)

    restored += convert_dec_to_hex(r1)
    restored += convert_dec_to_hex(r2)
    restored += convert_dec_to_hex(r3)
    restored += convert_dec_to_hex(r4)

print("복호화된 값:", restored)
print("원본과 같나?", restored == text)


복호화된 값: 0123456789abcdef0123456789abcdef
원본과 같나? True
